# Web Scraping - Strava

In [104]:
## Biblioteca necessárias

import pandas as pd
from time import sleep
from selenium import webdriver 
from selenium.webdriver.chrome.options import Options 
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys 

from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.action_chains import ActionChains

from selenium.webdriver.common.actions.action_builder import ActionBuilder
from selenium.webdriver.common.actions.mouse_button import MouseButton
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import selenium.common.exceptions
from selenium.common.exceptions import TimeoutException, NoSuchElementException,StaleElementReferenceException


## Iniciando uma instância do google chrome

In [105]:
# Utilizado para iniciar uma instacia do google chrome webdriver
service = ChromeDriverManager().install() # Pega o caminho do executavel chrome

#Usado para definir a preferência para o browser do chrome
options = webdriver.ChromeOptions()

#Inicia-se uma instância do google chrome com as definições 
driver = webdriver.Chrome(service, options=options)

sleep(1)

### Entrando no Site - Strava
url = 'https://www.strava.com/segments'
url_2 = 'https://www.strava.com/segments/35231066' # Segmento lamision35km2023
driver.get(url)

#Login 
usuario="joaomarcos1296@gmail.com"
senha="Jo@o1296"


campo_usuario = WebDriverWait(driver, 60).until(
    EC.presence_of_element_located(('xpath','//*[@id="email"]')))
campo_usuario.send_keys(usuario)

campo_senha = WebDriverWait(driver, 60).until(
    EC.presence_of_element_located(('xpath','//*[@id="password"]')))
campo_senha.send_keys(senha)

botao_login = WebDriverWait(driver, 60).until(
    EC.presence_of_element_located(('xpath','//*[@id="login-button"]'))) 
botao_login.click()
sleep(5)


# Acessa a Url Alvo
driver.get(url_2)
sleep(3)

botao_filtro_H_M = WebDriverWait(driver, 60).until(
    EC.presence_of_element_located(('xpath','//*[@id="segment-results"]/div[2]/table/tbody/tr/td[4]/div/button'))) 
botao_filtro_H_M.click()

botao_filtra_H = WebDriverWait(driver, 60).until(
    EC.presence_of_element_located(('xpath','//*[@id="segment-results"]/div[2]/table/tbody/tr/td[4]/div/ul/li[2]/a'))) 
botao_filtra_H.click()
sleep(3)


In [8]:
# Selecionando os dados dos corredores
s = 25
n = 26
for i in range(1,6):
        
    # Entrando nos resultados de cada corredor
    for j in range(s,n):
        try: # Verificando em qual aba está - 1 a 5 - caso não esteja na aba certa, clica-se na aba correta
            driver.find_element(
                by=By.XPATH,
                value=f'//*[@id="results"]/nav/ul/li[{i+1}]/a',
                ).click()
            sleep(2)
            driver.find_element(
                by = By.XPATH,
                value = f'//*[@id="results"]/table/tbody/tr[{j}]/td[3]/a',
                ).click() # Clicando no link do atleta
            driver.back()
            sleep(2)
        
        except Exception: # Caso ja esteja na aba correta, vai direto pro atleta
            try:
                driver.find_element(
                    by = By.XPATH,
                    value = f'//*[@id="results"]/table/tbody/tr[{j}]/td[3]/a',
                    ).click() # Clicando no link do atleta
                driver.back()
                sleep(2)
            except Exception:
                break



In [133]:
def verifica_nomes_atletas():
    try:
        # Tente encontrar um elemento que contenha os nomes dos atletas
        driver.find_element(By.XPATH, '//*[@id="results"]/table/tbody/tr/td[2]/a')
        return True  # Se encontrar, retorna True (há nomes de atletas)
    except NoSuchElementException:
        return False 

def verifica_num_pag():
    try:
        num_paginas = len(driver.find_elements(By.XPATH, '//*[@id="results"]/nav/ul/li'))
        if num_paginas == 0:
            num_paginas = 1
        else:
            num_paginas
    except:
        num_paginas = 1
    
    return num_paginas


def pega_nomes_atletas(nomes_atletas):
    ver = verifica_nomes_atletas()
    print(ver)
    pag = verifica_num_pag()
    print(pag)
    if ver:
        for a in range(1,pag+1):
            for n in range(1,26):          
                try:  
                    nome = driver.find_element(
                                by=By.XPATH,
                                value=f'//*[@id="results"]/table/tbody/tr[{n}]/td[2]/a',
                            ).text
                    nomes_atletas.append(nome)
                    print(nome)
                    #sleep(2)
                except Exception:
                    break
            try:
                print(a)
                print(f'//*[@id="results"]/nav/ul/li[{a+2}]/a')
                driver.find_element(
                    by=By.XPATH,
                    value=f'//*[@id="results"]/nav/ul/li[{a+2}]/a',
                    ).click()
                sleep(5)
                #a = a+1
            except NoSuchElementException:
                # Se não houver próxima página, saia do loop
                #a = 10
                break
    else:
        print("sem nomes")




In [134]:
nomes_atletas = []
pega_nomes_atletas(nomes_atletas)

True
1
1
//*[@id="results"]/nav/ul/li[3]/a


In [128]:
# Selecionando nomes e faixa de idade dos corredore
def pega_faixas_nomes(faixas_etarias, nomes_atletas):
    for i in range(2,11):
        # Clica no expandir as idades
        driver.find_element(
            by = By.XPATH,
            value = '//*[@id="premium-enhanced"]/ul/ul[2]/li[11]/button'
        ).click()
        sleep(2)

        # Clica na faixa idade
        #print(f'//*[@id="premium-enhanced"]/ul/ul[2]/li[{i}]/a')
        driver.find_element(
            by = By.XPATH,
            value = f'//*[@id="premium-enhanced"]/ul/ul[2]/li[{i}]/a'
        ).click()
        # Pega o texto da faixa etaria
        faixa_texto = driver.find_element(
            by = By.XPATH,
            value = f'//*[@id="premium-enhanced"]/ul/ul[2]/li[{i}]/a'
        ).text
        print(faixa_texto)
        faixas_etarias.append(faixa_texto)
        sleep(3)        
        # Pega o nome do atleta
        pega_nomes_atletas(nomes_atletas)
        
        sleep(2)



In [129]:
faixas_etarias = []
nomes_atletas = []

pega_faixas_nomes(faixas_etarias,nomes_atletas)
#pega_nomes_atletas(nomes_atletas)

False
1
sem nomes
True
1
1
//*[@id="results"]/nav/ul/li[3]/a
True
1
1
//*[@id="results"]/nav/ul/li[3]/a
True
4
1
//*[@id="results"]/nav/ul/li[3]/a
Anderson Batata
Paty Suellen
Luis O R Ratto
Cris Pescarmona ツ
Lillian Oliveira
Aquaria Mundi
Tony Ortiz
Rebeka Oppitz
CRISTIANO MORSCHBACHER
Tiago Schmidt da Silva
João Paulo Souza
Mara Silva
Eduardo Dudu
Lilian L. Andrade
Danilo Patricio Barbosa
Arthur Nagae
Elisângela Nogueira
GUTO 🇧🇷#NFG🇧🇷
Ana Jugurta
Evelyn Carboneiro
Káh Vieira
Robson Ribeiro
2
//*[@id="results"]/nav/ul/li[4]/a
True
1
1
//*[@id="results"]/nav/ul/li[3]/a
True
1
1
//*[@id="results"]/nav/ul/li[3]/a
False
1
sem nomes
False
1
sem nomes
False
1
sem nomes


In [120]:
len(nomes_atletas)
len(faixas_etarias)

9

In [121]:
nomes_atletas

['Graci Lindenmayr #WRTrail',
 'Mariana De Giovanni',
 'Henrique Reis',
 'Michel Morais',
 'Larissa Rodrigo',
 'Nicolas Queyroux',
 'Patricia Furtado',
 'Jose Juliano Lourenço',
 'Mariana Trindade',
 'Ana Carol Cassinelli',
 'Arthur Nagae',
 'Elisângela Nogueira',
 'GUTO 🇧🇷#NFG🇧🇷',
 'Ana Jugurta',
 'Evelyn Carboneiro',
 'Káh Vieira',
 'Robson Ribeiro']

In [52]:
df_teste = pd.DataFrame(nomes_atletas, faixas_etarias)
df_teste.head()

ValueError: Shape of passed values is (6, 1), indices imply (9, 1)

### Segments

In [12]:

## Procura Segmentos Alvo
segmento = "La Mision"

campo_seg = WebDriverWait(driver, 60).until(
    EC.presence_of_element_located(('xpath','//*[@id="keywords"]')))
campo_seg.send_keys(segmento)

botao_run = WebDriverWait(driver, 60).until(
    EC.presence_of_element_located(('xpath','//*[@id="activity-type"]/ul/li[2]/a'))) 
botao_run.click()

botao_busca = WebDriverWait(driver, 60).until(
    EC.presence_of_element_located(('xpath','//*[@id="climb-search-button"]'))) 
botao_busca.click()


